In [2]:
from typing import Annotated, Literal

from pydantic import BaseModel, Discriminator, Field, Tag


class Cat(BaseModel):
    pet_type: Literal['cat']
    age: int


class Dog(BaseModel):
    pet_kind: Literal['dog']
    age: int


def pet_discriminator(v):
    if isinstance(v, dict):
        return v.get('pet_type', v.get('pet_kind'))
    return getattr(v, 'pet_type', getattr(v, 'pet_kind', None))


class Model(BaseModel):
    pet: Annotated[Cat, Tag('cat')] | Annotated[Dog, Tag('dog')] = Field(
        discriminator=Discriminator(pet_discriminator)
    )


print(repr(Model.model_validate({'pet': {'pet_type': 'cat', 'age': 12}})))
#> Model(pet=Cat(pet_type='cat', age=12))

print(repr(Model.model_validate({'pet': {'pet_kind': 'dog', 'age': 12}})))

Model.model_json_schema()

#> Model(pet=Dog(pet_kind='dog', age=12))

Model(pet=Cat(pet_type='cat', age=12))
Model(pet=Dog(pet_kind='dog', age=12))


{'$defs': {'Cat': {'properties': {'pet_type': {'const': 'cat',
     'title': 'Pet Type',
     'type': 'string'},
    'age': {'title': 'Age', 'type': 'integer'}},
   'required': ['pet_type', 'age'],
   'title': 'Cat',
   'type': 'object'},
  'Dog': {'properties': {'pet_kind': {'const': 'dog',
     'title': 'Pet Kind',
     'type': 'string'},
    'age': {'title': 'Age', 'type': 'integer'}},
   'required': ['pet_kind', 'age'],
   'title': 'Dog',
   'type': 'object'}},
 'properties': {'pet': {'oneOf': [{'$ref': '#/$defs/Cat'},
    {'$ref': '#/$defs/Dog'}],
   'title': 'Pet'}},
 'required': ['pet'],
 'title': 'Model',
 'type': 'object'}

In [3]:

from dataclasses import dataclass, asdict, fields

@dataclass
class Product:
    product_id: int
    name: str
    internal_notes: str  # Field to be excluded
    price: float
    _client: str

def excluding_dict_factory(data, excluded_fields):
    """
    A dict_factory that excludes specified fields.
    """
    return dict(item for item in data if item[0] not in excluded_fields)

# Usage
product = Product(product_id=101, name="Laptop", _client="blah", internal_notes="High margin item", price=1200.00)
excluded = {"_client"} # Use a set for efficient lookups

# We need to wrap the factory to pass the excluded fields
from functools import partial
custom_factory = partial(excluding_dict_factory, excluded_fields=excluded)

product_dict = asdict(product, dict_factory=custom_factory)

print(product_dict)

{'product_id': 101, 'name': 'Laptop', 'internal_notes': 'High margin item', 'price': 1200.0}


In [10]:

from pydantic import BaseModel, Field, ConfigDict
from pywa.types import Button, SectionList


class TextResponse(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    content: str | None = Field(description="Content of the response.")
    buttons: list[Button] = Field(
        default=[],
        description="Buttons that can be added to the response. Should be left empty unless specified.",
    )
    section_list: SectionList | None = Field(
        default=None, description="Section list with multiple choice options. Should be left null unless specified."
    )

response = TextResponse(content="blah", buttons=[Button("do", "something")])
response


response.model_json_schema()

{'$defs': {'Button': {'properties': {'title': {'title': 'Title',
     'type': 'string'},
    'callback_data': {'title': 'Callback Data', 'type': 'string'}},
   'required': ['title', 'callback_data'],
   'title': 'Button',
   'type': 'object'},
  'Section': {'properties': {'title': {'title': 'Title', 'type': 'string'},
    'rows': {'items': {'$ref': '#/$defs/SectionRow'},
     'title': 'Rows',
     'type': 'array'}},
   'required': ['title', 'rows'],
   'title': 'Section',
   'type': 'object'},
  'SectionList': {'properties': {'button_title': {'title': 'Button Title',
     'type': 'string'},
    'sections': {'items': {'$ref': '#/$defs/Section'},
     'title': 'Sections',
     'type': 'array'}},
   'required': ['button_title', 'sections'],
   'title': 'SectionList',
   'type': 'object'},
  'SectionRow': {'properties': {'title': {'title': 'Title', 'type': 'string'},
    'callback_data': {'title': 'Callback Data', 'type': 'string'},
    'description': {'anyOf': [{'type': 'string'}, {'type'